<a href="https://colab.research.google.com/github/lautaromartinez8964/my_rep/blob/main/%E6%AF%95%E8%AE%BE2%E5%89%AF%E6%9C%AC_robust_z%E7%9B%91%E6%B5%8B%E6%B4%AA%E6%B0%B4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install geemap
%pip install pygis

import ee
import geemap

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='geemap-441216')

从谷歌云盘里加载z_flood_robust文件夹

In [2]:
from google.colab import drive
drive.mount('/content/drive')

import sys
sys.path.append('/content/drive/MyDrive/Colab_Notebooks/')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
try:
    import z_flood_robust
    # 尝试调用 z_flood 中的函数
    # 这里假设你已经在 __init__.py 中导入了 calc_basemean
    # 如果没有，你需要使用 z_flood.zscore.calc_basemean
    z_flood_robust.calc_basemad
    print("z_flood 包已成功导入，并且可以访问其中的函数")
except AttributeError:
    print("z_flood 包可能已导入，但无法访问其中的函数/变量")
except ImportError:
     print("导入z_flood失败")

z_flood 包已成功导入，并且可以访问其中的函数


# 1.导入依赖

In [4]:
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle #用于在图表中添加形状(例如矩形)
import re #正则表达式模块
from z_flood_robust import calc_basemedian, calc_basemad, calc_median_anomaly, calc_robust_zscore, calc_basemean,calc_basestd,calc_zscore
from z_flood_robust import mapFloods,floodPalette

from ipywidgets import Label #用于创建交互式控件

# 2.定义交互式地图以选择感兴趣区域

In [5]:
#解析用户点击地图时生成的坐标字符串，提取经纬度并返回
def parseClickedCoordinates(label):
  #利用正则表达式，从label.value提取出坐标：一个可能带有负号的浮点数 这里为【经度，纬度】
  #正则表达式： r'(?:-)?[0-9]+.[0-9]+'
  #r'表示原始字符，(?:-)是非捕获组，?:表示负号是可选的，[0-9]+表示匹配一个数字，+表示可以出现一次或多次，.匹配小数点，最后再次匹配一个或多个数字
  coords = [float(c) for c in re.findall(r'(?:-)?[0-9]+.[0-9]+', label.value)]
  coords.reverse() #反转为【纬度，经度】，符合GEE坐标格式
  return coords

#创建一个Lable，用于显示用户点击的目标
l = Label()
display(1)

1

In [6]:
#处理用户与地图的交互事件，当用户点击地图时，将点击的坐标存储到Label控件中
def handle_interaction(**kwargs):
  #kwargs包含交互事件的参数 kwargs.get('type'):获取事件类型为鼠标点击 kwargs.get('coordinates')：获取点击的坐标，转换为字符串并存储到Label控件中
  if kwargs.get('type') == 'click':
    l.value = str(kwargs.get('coordinates'))

#创建交互式地图
Map = geemap.Map()
Map.on_interaction(handle_interaction)
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

# 3.定义几何范围并展示

In [9]:
lon,lat = parseClickedCoordinates(l)
w,h = 0.1,0.1 #矩形宽度与高度（单位：度）

geometry = ee.Geometry.Polygon(
    [[[lon-w,lat-h],
     [lon-w,lat+h],
     [lon+w,lat+h],
     [lon+w,lat-h]]]
)

#将几何范围添加到地图
Map.addLayer(
    geometry,
    {'color':'red','fillColor':'00000000'},
    'AOI'
)
Map

Map(bottom=199852.0, center=[39.436192999314095, -0.44445008807827874], controls=(WidgetControl(options=['posi…

# 4.过滤Sentinel-1数据

以下是西班牙巴伦西亚区域

In [10]:
targdate = '2024-11-01'   # 目标日期 (需要根据实际数据可用性验证或微调)
basestart = '2024-01-20'  # 基线开始日期
baseend = '2024-09-20'   # 基线结束日期

# !!! (可选) 替换硬编码的 aoi2 !!!
# 如果你决定使用固定的 valencia_aoi，在这里替换：
aoi3 = ee.Geometry.Rectangle([-0.55, 39.35, -0.30, 39.50]) # 新的巴伦西亚AOI
# !!! 更好的做法是完全不用 aoi2，下面 filterBounds 直接用 analysis_aoi !!!

filters = [
    ee.Filter.listContains("transmitterReceiverPolarisation", "VV"),
    # ee.Filter.listContains("transmitterReceiverPolarisation", "VH"), # 如果需要VH也取消注释
    ee.Filter.equals("instrumentMode", "IW"),
    # !!! 重要：检查轨道方向 !!!
    # 需要确认 targdate 前后过境巴伦西亚的 S1 影像是升轨还是降轨
    # 可能需要改为 'DESCENDING'，或不加这个过滤器以获取所有轨道
    ee.Filter.equals("orbitProperties_pass", "ASCENDING"), # <--- 可能需要修改或移除!
    # 时间过滤器现在是动态的，覆盖基线期和目标期，无需修改
    ee.Filter.date('2023-10-01', ee.Date(targdate).advance(1, 'day'))
]

# 加载S1数据并计算Z分数
s1 = ee.ImageCollection("COPERNICUS/S1_GRD") \
    .filter(filters) \
    .filterBounds(aoi3) # !!! 使用统一的 analysis_aoi !!!



# 检查是否有影像满足条件
s1_size = s1.size().getInfo()
print(f"找到满足条件的 Sentinel-1 影像数量: {s1_size}")
if s1_size == 0:
    print("警告：未找到满足条件的影像，请检查时间范围、AOI 或轨道方向过滤器！")
print(s1.aggregate_array('system:time_start').map(lambda t: ee.Date(t).format()).getInfo())

# Z 分数计算 (轨道方向参数需要与上面 filter 保持一致)
# orbit_direction = "ASCENDING" # <--- 如果上面修改了这里也要改
orbit_direction = "ASCENDING" if ee.Filter.equals("orbitProperties_pass", "ASCENDING") in filters else "DESCENDING" #或者更灵活的方式
z = calc_zscore(s1, basestart, baseend, 'IW', orbit_direction)
rz = calc_robust_zscore(s1, basestart, baseend, 'IW', orbit_direction)

找到满足条件的 Sentinel-1 影像数量: 67
['2023-10-01T18:02:59', '2023-10-08T17:55:04', '2023-10-13T18:02:59', '2023-10-20T17:55:05', '2023-10-25T18:02:59', '2023-11-01T17:55:04', '2023-11-06T18:02:59', '2023-11-13T17:55:04', '2023-11-18T18:02:59', '2023-11-25T17:55:03', '2023-11-30T18:02:58', '2023-12-07T17:55:03', '2023-12-12T18:02:58', '2023-12-19T17:55:02', '2023-12-24T18:02:57', '2023-12-31T17:55:02', '2024-01-05T18:02:56', '2024-01-12T17:55:01', '2024-01-17T18:02:56', '2024-01-24T17:55:01', '2024-01-29T18:02:56', '2024-02-05T17:55:00', '2024-02-10T18:02:55', '2024-02-17T17:55:00', '2024-02-22T18:02:55', '2024-02-29T17:55:00', '2024-03-05T18:02:55', '2024-03-12T17:55:00', '2024-03-17T18:02:55', '2024-03-24T17:55:01', '2024-03-29T18:02:56', '2024-04-05T17:55:01', '2024-04-10T18:02:55', '2024-04-17T17:55:01', '2024-04-22T18:02:56', '2024-04-29T17:55:02', '2024-05-04T18:02:57', '2024-05-11T17:55:02', '2024-05-16T18:02:57', '2024-05-23T17:55:02', '2024-05-28T18:02:56', '2024-06-04T17:55:01', '2024

In [ ]:
# 定义一个函数，将日期信息添加到每个图像
def addDate(image):
  date = image.date().millis() #获取日期毫秒表示
  return image.addBands(ee.Image.constant(date).rename('date').long())
#将函数应用到图像集和中每个图像
s1_with_date = s1.map(addDate)
mosaic_image = s1_with_date.mosaic().clip(aoi3)
Map.addLayer(mosaic_image.select('date'), {'min': ee.Date('2015-01-01').millis(), 'max': ee.Date(targdate).millis()}, 'Date (milliseconds)')
Map

# 5.显示Z分数图层

In [11]:
Map = geemap.Map()
Map.setCenter(lon,lat,11)
#{0}表示占位符，表示在字符串该位置中插入一个变量(将目标日期格式化)
Map.addLayer(s1.mosaic().clip(aoi3).select('VV'), {'min': -25, 'max': 0}, 'VV Backscatter (dB); {0}'.format(targdate))

#z分数的颜色
zpalette = ['#b2182b','#ef8a62','#fddbc7','#f7f7f7','#d1e5f0','#67a9cf','#2166ac']
clipped_rz = rz.map(lambda image: image.clip(geometry))
clipped_z = z.map(lambda image: image.clip(geometry))
Map.addLayer(clipped_rz.mosaic().select('VV'), {'min': -5, 'max': 5, 'palette': zpalette}, 'VV RZ-score; {0}'.format(targdate))
Map.addLayer(clipped_z.mosaic().select('VV'), {'min': -5, 'max': 5, 'palette': zpalette}, 'VV Z-score; {0}'.format(targdate))


#用于点击地图某一点，生成Z分数时间序列图的点击函数(点击地图传递坐标)
label = Label()
def handle_interaction(**kwargs):
  if kwargs.get('type') == 'click':
    coords = kwargs.get('coordinates')
    label.value = str(coords)

Map.on_interaction(handle_interaction)
Map


Map(center=[39.43671866405075, -0.43742784684509767], controls=(WidgetControl(options=['position', 'transparen…

In [12]:
#获取感兴趣区域（用于论文绘图)
roi_choose1 = Map.user_roi

if roi_choose1 is not None:
  #获取ROI类型
  roi_type = roi_choose1.type().getInfo()
  print(f"ROI 类型：{roi_type}")

  #如果是Polygon，获取坐标
  if roi_type == 'Polygon':
    coords = roi_choose1.coordinates().getInfo()
    print(f"ROI 坐标：{coords}")

ROI 类型：Polygon
ROI 坐标：[[[-0.419407, 39.336948], [-0.419407, 39.452096], [-0.347953, 39.452096], [-0.347953, 39.336948], [-0.419407, 39.336948]]]
